### This jupyter notebook is devoted to microbatch and train_mode. Notebook also shown how to use multiple GPUs. 

Import Libraries.
Specify which GPU(s) to be used.

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import warnings
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='4,5,6,7' 

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

sys.path.append('../..')
from batchflow import Pipeline, B, C, V, D
from batchflow.opensets import MNIST
from batchflow.models.tf import VGG7

In [2]:
BATCH_SIZE = 64

# Create a dataset

[MNIST](http://yann.lecun.com/exdb/mnist/) is a dataset of handwritten digits frequently used as a baseline for machine learning tasks.

Downloading MNIST database might take a few minutes to complete.

A dataset is defined by an index (a sequence of item ids) and a batch class (see [the documentation for details](https://analysiscenter.github.io/batchflow/intro/dataset.html)).

In [3]:
dataset = MNIST(bar=True)

100%|██████████| 8/8 [00:01<00:00,  3.04it/s]


# Define a pipeline config

Config allows to create flexible pipelines which take parameters.

For instance, if you put a model type into config, you can run a pipeline against different models.

See [a list of available models](https://analysiscenter.github.io/batchflow/intro/tf_models.html#ready-to-use-models) to choose the one which fits you best.

In [4]:
config = dict(model=VGG7)

**Microbatching** allows to process given data sequentially, accumulating gradients from microbatches and applying them once in the end. The size of the microbatch can be specified in two places; the value that was specified last will be used.
<span style="color:red">**Microbatch size must be a divisor of the batch size!**</span>

In [5]:
MICROBATCH_SIZE = 32

*scope* – subset of variables to optimize during training. Can be either string or sequence of strings.
Value ```''``` is reserved for optimizing all trainable variables. Putting ```-``` sign before name stands for complement: optimize everything but the passed scope.

*train_steps* – configuration of different training procedures. It allows to optimize parametrs of selected scope using selected optimizer, loss, decay. Optimizer and decay may be reused by another *train_step*. Use **train_mode** to select *train_step* and fetch loss according to this *train_step*. Watch [FreezeOut](https://arxiv.org/abs/1706.04983) to find out what it is for.

**device** – allow train model on multiple GPU (Сreates a copy of model on each selected GPU). Initialization of large model on a large number of GPU may take some time (minuts or tens of minutes)!

Example:
```python
'device': 'GPU:*'            # Used all avalible GPU
'device': ['GPU:0', 'GPU:1'] # Used GPU:0 and GPU:1
```
<span style="color:red">**Number of devices must be a divisor of the batch size! (If microbathing ~~batch size~~ microbatch size)**</span>

In [6]:
lr_decay_config = ('exp', {'learning_rate': 0.005,
                           'decay_steps': 100,
                           'decay_rate': 0.99})

model_config = {'inputs': {'images/shape': (B('image_shape')),
                           'labels': {'classes': D('num_classes'),
                                      'transform': 'ohe'}},
                'initial_block': {'inputs': 'images'},
                'microbatch': MICROBATCH_SIZE,
                'train_steps': {'all': {'optimizer': 'Adam'},
                                'all_with_decay': {'optimizer': 'Adam', 'decay': lr_decay_config},
                                'custom': {'optimizer': 'RMSProp', 'scope': '-body/group-0', 'decay': lr_decay_config},
                                'part_head': {'use': 'all', 'scope': 'head/layer-2', 'loss': 'mse'}
                               }
               }

# Create a template pipeline

In [7]:
train_template = (Pipeline(config=config)
                  .to_array()
                  .train_model(name='conv_nn',
                               use_lock=True,
                               fetches='loss_all', 
                               images=B('images'), 
                               labels=B('labels'),
                               save_to=V('current_loss'), 
                               train_mode='all')
                  .update_variable('loss_history', V('current_loss'), mode='a'))

In [8]:
(train_template.before
 .init_variable('loss_history', init_on_each_run=list)
 .init_variable('current_loss')
 .init_model(mode='dynamic',
             model_class=C('model'),
             name='conv_nn',
             config=model_config))

# Train the model

Apply a dataset to a template pipeline to create a runnable pipeline:

In [9]:
train_pipeline = (train_template << dataset.train)

In [10]:
train_pipeline.run(BATCH_SIZE, shuffle=True, n_epochs=1, bar='n', drop_last=True)

# Test the model

In [11]:
test_pipeline = (Pipeline()
                 .import_model(model='conv_nn', 
                               pipeline=train_pipeline)
                 .to_array()
                 .predict_model(name='conv_nn',
                                fetches='predictions', 
                                images=B('images'), 
                                save_to=V('predictions'))
                 .gather_metrics('class', targets=B('labels'), predictions=V('predictions'),
                                 fmt='logits', axis=-1, save_to=V('metrics', mode='a')))

In [12]:
(test_pipeline.before
 .init_variable('predictions') 
 .init_variable('metrics', init_on_each_run=None))

In [13]:
test_pipeline = (test_pipeline << dataset.test)

In [14]:
test_pipeline.run(BATCH_SIZE, shuffle=True, n_iters=3, bar='n', prefetch=0)

Let's get the accumulated [metrics information](https://analysiscenter.github.io/batchflow/intro/models.html#model-metrics)

In [15]:
metrics = test_pipeline.get_variable('metrics')

In [16]:
metrics.evaluate('accuracy')

0.9895833333333334